In [1]:
import re
import math
from pathlib import Path
from typing import Dict, List, Tuple
import pandas as pd
import numpy as np

In [ ]:
def _last_block(pattern: str, text: str) -> str:
"""返回最后一次匹配的区块字符串。pattern 应尽量包含起止锚点。"""
matches = list(re.finditer(pattern, text, flags=re.S))
return matches[-1].group(0) if matches else ""


# --- NPA charges ---
NPA_HDR = r"Summary of Natural Population Analysis:(.*?)\n\s*Sum of atomic charges" # 截止到总和
NPA_LINE = re.compile(r"^\s*(\d+)\s+([A-Za-z]{1,2})\s+([-+]?\d+\.\d+)", re.M)


# --- Standard orientation table ---
STD_ORI = r"Standard orientation:(.*?)\n\s*-+\n\s*Center\s+Atomic\s+Atomic\s+X\s+Y\s+Z(.*?)\n\s*-+" # 捕获表体
STD_LINE = re.compile(r"^\s*(\d+)\s+(\d+)\s+\d+\s+([-+]?\d+\.\d+)\s+([-+]?\d+\.\d+)\s+([-+]?\d+\.\d+)", re.M)


# --- Wiberg bond index ---
WBI_HDR = r"Wiberg bond index(.*?)\n\s*\-\s*\n" # 视版本改进
WBI_LINE = re.compile(r"^\s*(\d+)\s+(\d+)\s+([-+]?\d+\.\d+)", re.M) # 若打印为三列列表


# 注：不同 Gaussian/NBO 版本 WBI 可能为矩阵或三列列表；必要时新增矩阵解析器